In [1]:
import requests, json, time, re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt

In [2]:
# create variables of strings for the different subreddit pages
sub_all = '/top.json?t=all'
sub_gilded = '/gilded.json'
sub_hot = '.json'
sub_new = '/new.json'

In [4]:
# adapted code for Adi
# posts is the returned dictionary (should pass in previously ran `posts`)
# subreddit = url string 
# subpage = the page on the subreddit (should be already defined)
def sub_query(subreddit, subpage, posts=None, pages=60):
    
    #Step 0: set initial conditions
    if posts == None:
        posts = []
    else:
        posts = posts
    url = f'https://old.reddit.com/r/{subreddit}{subpage}'
    after = None

    #Step 1: build the loop and print URL
    for _ in range(pages):
        if after == None:
            current_url = url
        elif subpage == '/top.json?t=all':
            current_url = url + '&after=' + after
        else:
            current_url = url + '?count=25&after=' + after

        print(current_url)

        #Step 2a: make the requests and handle status code. Add time.sleep now (2.b)
        res = requests.get(current_url, headers={'User-agent': 'hehehehe'})

        if res.status_code != 200:
            print('Status error', res.status_code)
            break

        #Step 3: Actually deal with the data   
        current_dict = res.json()
        current_posts = [p['data'] for p in current_dict['data']['children']]
        posts.extend(current_posts)

        after = current_dict['data']['after']

        #Step 2b:
        time.sleep(0.1)

    return posts

In [5]:
# r/news all
news = sub_query('news', sub_all)

https://old.reddit.com/r/news/top.json?t=all
https://old.reddit.com/r/news/top.json?t=all&after=t3_aezedn
https://old.reddit.com/r/news/top.json?t=all&after=t3_7aqilp
https://old.reddit.com/r/news/top.json?t=all&after=t3_925h9r
https://old.reddit.com/r/news/top.json?t=all&after=t3_7ib4bo
https://old.reddit.com/r/news/top.json?t=all&after=t3_afiri2
https://old.reddit.com/r/news/top.json?t=all&after=t3_9zop96
https://old.reddit.com/r/news/top.json?t=all&after=t3_73n05s
https://old.reddit.com/r/news/top.json?t=all&after=t3_77iqr8
https://old.reddit.com/r/news/top.json?t=all&after=t3_a5zya6
https://old.reddit.com/r/news/top.json?t=all&after=t3_8vjnng
https://old.reddit.com/r/news/top.json?t=all&after=t3_aqy9ba
https://old.reddit.com/r/news/top.json?t=all&after=t3_b31o1f
https://old.reddit.com/r/news/top.json?t=all&after=t3_7zwi9g
https://old.reddit.com/r/news/top.json?t=all&after=t3_6wj9ey
https://old.reddit.com/r/news/top.json?t=all&after=t3_6z3755
https://old.reddit.com/r/news/top.json?t

In [6]:
# r/news hot
news = sub_query('news', sub_hot, news)

https://old.reddit.com/r/news.json
https://old.reddit.com/r/news.json?count=25&after=t3_cc79gm
https://old.reddit.com/r/news.json?count=25&after=t3_cc1fwt
https://old.reddit.com/r/news.json?count=25&after=t3_cc3t2e
https://old.reddit.com/r/news.json?count=25&after=t3_cbpziu
https://old.reddit.com/r/news.json?count=25&after=t3_cbvsc0
https://old.reddit.com/r/news.json?count=25&after=t3_cblfem
https://old.reddit.com/r/news.json?count=25&after=t3_cbmvyz
https://old.reddit.com/r/news.json?count=25&after=t3_cb55a8
https://old.reddit.com/r/news.json?count=25&after=t3_caqvkm
https://old.reddit.com/r/news.json?count=25&after=t3_carltg
https://old.reddit.com/r/news.json
https://old.reddit.com/r/news.json?count=25&after=t3_cc79gm
https://old.reddit.com/r/news.json?count=25&after=t3_cc1fwt
https://old.reddit.com/r/news.json?count=25&after=t3_cc3t2e
https://old.reddit.com/r/news.json?count=25&after=t3_cbpziu
https://old.reddit.com/r/news.json?count=25&after=t3_cbvsc0
https://old.reddit.com/r/news.

In [7]:
# r/news new
news = sub_query('news', sub_new, news)

https://old.reddit.com/r/news/new.json
https://old.reddit.com/r/news/new.json?count=25&after=t3_cc5k6x
https://old.reddit.com/r/news/new.json?count=25&after=t3_cc2lew
https://old.reddit.com/r/news/new.json?count=25&after=t3_cbz8mq
https://old.reddit.com/r/news/new.json?count=25&after=t3_cbvs5j
https://old.reddit.com/r/news/new.json?count=25&after=t3_cbmzk8
https://old.reddit.com/r/news/new.json?count=25&after=t3_cbidx9
https://old.reddit.com/r/news/new.json?count=25&after=t3_cbebnm
https://old.reddit.com/r/news/new.json?count=25&after=t3_cb9e71
https://old.reddit.com/r/news/new.json?count=25&after=t3_cb53wm
https://old.reddit.com/r/news/new.json?count=25&after=t3_cb0hws
https://old.reddit.com/r/news/new.json?count=25&after=t3_carsar
https://old.reddit.com/r/news/new.json
https://old.reddit.com/r/news/new.json?count=25&after=t3_cc5k6x
https://old.reddit.com/r/news/new.json?count=25&after=t3_cc2lew
https://old.reddit.com/r/news/new.json?count=25&after=t3_cbz8mq
https://old.reddit.com/r/n

In [9]:
theO = sub_query('theonion', sub_all)

https://old.reddit.com/r/theonion/top.json?t=all
https://old.reddit.com/r/theonion/top.json?t=all&after=t3_8y0ob1
https://old.reddit.com/r/theonion/top.json?t=all&after=t3_a3qyte
https://old.reddit.com/r/theonion/top.json?t=all&after=t3_b9ekqj
https://old.reddit.com/r/theonion/top.json?t=all&after=t3_bo676d
https://old.reddit.com/r/theonion/top.json?t=all&after=t3_aqp4sb
https://old.reddit.com/r/theonion/top.json?t=all&after=t3_airdvw
https://old.reddit.com/r/theonion/top.json?t=all&after=t3_9r2707
https://old.reddit.com/r/theonion/top.json?t=all&after=t3_b3by5b
https://old.reddit.com/r/theonion/top.json?t=all&after=t3_9f8m59
https://old.reddit.com/r/theonion/top.json?t=all&after=t3_av1jt0
https://old.reddit.com/r/theonion/top.json?t=all&after=t3_9xo00e
https://old.reddit.com/r/theonion/top.json?t=all&after=t3_alilh7
https://old.reddit.com/r/theonion/top.json?t=all&after=t3_9hrc6t
https://old.reddit.com/r/theonion/top.json?t=all&after=t3_aqxwzy
https://old.reddit.com/r/theonion/top.jso

In [10]:
theO = sub_query('theonion', sub_hot, theO)

https://old.reddit.com/r/theonion.json
https://old.reddit.com/r/theonion.json?count=25&after=t3_c7sydx
https://old.reddit.com/r/theonion.json?count=25&after=t3_c5tuut
https://old.reddit.com/r/theonion.json?count=25&after=t3_c3eh8m
https://old.reddit.com/r/theonion.json?count=25&after=t3_c2jiw8
https://old.reddit.com/r/theonion.json?count=25&after=t3_c1etej
https://old.reddit.com/r/theonion.json?count=25&after=t3_c07iud
https://old.reddit.com/r/theonion.json?count=25&after=t3_bzfjqi
https://old.reddit.com/r/theonion.json?count=25&after=t3_bxxnp2
https://old.reddit.com/r/theonion.json?count=25&after=t3_bx557x
https://old.reddit.com/r/theonion.json?count=25&after=t3_bvlled
https://old.reddit.com/r/theonion.json?count=25&after=t3_butqeo
https://old.reddit.com/r/theonion.json?count=25&after=t3_btrf66
https://old.reddit.com/r/theonion.json?count=25&after=t3_bs6yfs
https://old.reddit.com/r/theonion.json?count=25&after=t3_bqyd4x
https://old.reddit.com/r/theonion.json?count=25&after=t3_bqjb9f
h

In [11]:
theO = sub_query('theonion', sub_new, theO)

https://old.reddit.com/r/theonion/new.json
https://old.reddit.com/r/theonion/new.json?count=25&after=t3_c80d79
https://old.reddit.com/r/theonion/new.json?count=25&after=t3_c5rqxz
https://old.reddit.com/r/theonion/new.json?count=25&after=t3_c3di33
https://old.reddit.com/r/theonion/new.json?count=25&after=t3_c2jaab
https://old.reddit.com/r/theonion/new.json?count=25&after=t3_c15ttr
https://old.reddit.com/r/theonion/new.json?count=25&after=t3_c0799s
https://old.reddit.com/r/theonion/new.json?count=25&after=t3_bzdxx6
https://old.reddit.com/r/theonion/new.json?count=25&after=t3_bxwhjf
https://old.reddit.com/r/theonion/new.json?count=25&after=t3_bwu2md
https://old.reddit.com/r/theonion/new.json?count=25&after=t3_bveamj
https://old.reddit.com/r/theonion/new.json?count=25&after=t3_buk2d6
https://old.reddit.com/r/theonion/new.json?count=25&after=t3_btngbz
https://old.reddit.com/r/theonion/new.json?count=25&after=t3_bs6yfs
https://old.reddit.com/r/theonion/new.json?count=25&after=t3_brcwg6
https

In [8]:
# combines the posts in the subreddit
def combine_text(posts):
    return[' '.join([post['title'], post['selftext']]) for post in posts] # build this out to a function

In [14]:
# combine all of news
news_text = combine_text(news)

In [15]:
# combine all theOnion
theO_text = combine_text(theO)

In [16]:
# set theOnion to 0
news_df = pd.DataFrame(news_text, columns=['text'])
news_df['is_theO']= 0

In [17]:
# set 
theO_df = pd.DataFrame(theO_text, columns=['text'])
theO_df['is_theO']= 1

In [18]:
reddit = pd.concat([news_df, theO_df], ignore_index=True)

In [19]:
reddit['is_theO'].value_counts()

1    4487
0    4406
Name: is_theO, dtype: int64

In [20]:
reddit.shape

(8893, 2)

In [21]:
reddit = reddit.drop_duplicates()

In [22]:
reddit.shape

(3058, 2)

In [23]:
reddit['is_theO'].value_counts(normalize=True)

1    0.580772
0    0.419228
Name: is_theO, dtype: float64

In [24]:
reddit.to_csv('./datasets/reddit_5.csv', index=False)